# Bring your own LLMs

Ragas uses langchain under the hood for connecting to LLMs for metrices that require them. This means you can swap out the default LLM we use (`gpt-3.5-turbo-16k`) to use any 100s of API supported out of the box with langchain.

- [Completion LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.llms)
- [Chat based LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chat_models)

This guide will show you how to use another or LLM API for evaluation.

## Evaluating with GPT-3.5-turbo-instruct

In [1]:
%pip show ragas

Name: ragas
Version: 0.0.15.dev0+ge194caa.d20230923
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /home/inflaton/miniconda3/envs/ragas/lib/python3.11/site-packages
Editable project location: /home/inflaton/code/gpt/ragas_extended
Requires: datasets, langchain, numpy, openai, pydantic, pysbd, sentence-transformers, transformers
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [4]:
pruned_index = [1,  2,  3,  7,  9, 10, 12, 13, 14, 15, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28]
pruned_ds = fiqa_eval["baseline"].select(pruned_index)
pruned_ds.to_pandas()

,question,ground_truths,answer,contexts
0,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...
1,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...
2,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...
3,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro..."
4,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...
5,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...",\nUsing credit card points to pay for tax dedu...,"[""For simplicity, let's start by just consider..."
6,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...",\nInvesting/business with other people's money...,"[""Basically, you either borrow money, or get o..."
7,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...",\nThere are several approaches for pricing a s...,"[I don't have any experience in this, but this..."
8,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,\nMoney earned and spent prior to establishing...,[Funds earned and spent before opening a dedic...
9,Do I need a new EIN since I am hiring employee...,[I called the IRS (click here for IRS contact ...,"\nNo, you do not need a new EIN since you are ...",[You don't need to notify the IRS of new membe...


In [5]:
pruned_ds.to_csv("pruned-ds.csv", sep='\t')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

100872

In [6]:
"""
Official evaluation script for QAConv, modified from SQuAD 2.0.

 * Copyright (c) 2021, salesforce.com, inc.
 * All rights reserved.
 * SPDX-License-Identifier: BSD-3-Clause
 * For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

"""

import collections
import re
import string


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


In [7]:
cut_off_at =  5 # dataset.num_rows
new_ds = pruned_ds.map(
    lambda record, idx: {
        "answer": record["ground_truths"][0] if idx < cut_off_at else  record["answer"],
        "EM": compute_exact(record['ground_truths'][0], record["ground_truths"][0] if idx < cut_off_at else record["answer"]), 
        "F1": compute_f1(record['ground_truths'][0], record["ground_truths"][0] if idx < cut_off_at else record["answer"])
    },
    batched=False,
    with_indices=True,
)
new_ds

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts', 'EM', 'F1'],
    num_rows: 20
})

In [8]:
new_ds.to_pandas()

,question,ground_truths,answer,contexts,EM,F1
0,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...,1,1.000000
1,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...,1,1.000000
2,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...","""I'm afraid the great myth of limited liabilit...",[Set up a meeting with the bank that handles y...,1,1.000000
3,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...","""If your sole proprietorship losses exceed all...","[""Short answer, yes. But this is not done thro...",1,1.000000
4,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1,1.000000
5,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...",\nUsing credit card points to pay for tax dedu...,"[""For simplicity, let's start by just consider...",0,0.199367
6,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...",\nInvesting/business with other people's money...,"[""Basically, you either borrow money, or get o...",0,0.356436
7,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...",\nThere are several approaches for pricing a s...,"[I don't have any experience in this, but this...",0,0.383495
8,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,\nMoney earned and spent prior to establishing...,[Funds earned and spent before opening a dedic...,0,0.671642
9,Do I need a new EIN since I am hiring employee...,[I called the IRS (click here for IRS contact ...,"\nNo, you do not need a new EIN since you are ...",[You don't need to notify the IRS of new membe...,0,0.331288


In [9]:
%%time
# evaluate
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)

result = evaluate(
    new_ds,
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

result

using model: gpt-3.5-turbo-instruct
evaluating with [faithfulness]


100%|██████████| 2/2 [00:10<00:00,  5.42s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [00:21<00:00, 10.82s/it]


CPU times: user 2.21 s, sys: 404 ms, total: 2.62 s
Wall time: 35.9 s


{'ragas_score': 0.8580, 'faithfulness': 0.8242, 'answer_relevancy': 0.8947}

In [10]:
df = result.to_pandas()
df

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,[Sure you can. You can fill in whatever you w...,1.000000,0.839067
1,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,[You're confusing a lot of things here. Compan...,1.000000,0.787693
2,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...","[""I'm afraid the great myth of limited liabili...",1.000000,0.827112
3,Intentions of Deductible Amount for Small Busi...,"[""Short answer, yes. But this is not done thro...","""If your sole proprietorship losses exceed all...","[""If your sole proprietorship losses exceed al...",1.000000,0.769419
4,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,[Depends whom the 1099 was issued to. If it wa...,1.000000,0.848734
5,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...",\nUsing credit card points to pay for tax dedu...,"[""For simplicity, let's start by just consider...",1.000000,0.934078
6,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...",\nInvesting/business with other people's money...,"[""Basically, you either borrow money, or get o...",1.000000,0.938330
7,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...",\nThere are several approaches for pricing a s...,"[I don't have any experience in this, but this...",1.000000,0.960979
8,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,\nMoney earned and spent prior to establishing...,[Funds earned and spent before opening a dedic...,1.000000,0.932541
9,Do I need a new EIN since I am hiring employee...,[You don't need to notify the IRS of new membe...,"\nNo, you do not need a new EIN since you are ...",[I called the IRS (click here for IRS contact ...,1.000000,0.915574


In [18]:
cut_off_at = 5
result_all = pruned_ds.map(
    lambda record, idx: {
        "EM": compute_exact(record['ground_truths'][0], record["ground_truths"][0] if idx < cut_off_at else  record["answer"]), 
        "F1": compute_f1(record['ground_truths'][0], record["ground_truths"][0] if idx < cut_off_at else  record["answer"]),
        "faithfulness (gpt-3.5-turbo-instruct)": df["faithfulness"][idx], 
        "answer_relevancy (gpt-3.5-turbo-instruct)": df["answer_relevancy"][idx], 
        "ragas_score (gpt-3.5-turbo-instruct)": 2 * df["faithfulness"][idx] * df["answer_relevancy"][idx] / (df["faithfulness"][idx] + df["answer_relevancy"][idx])
    },
    batched=False,
    with_indices=True,
    remove_columns=pruned_ds.column_names
)
result_all

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['EM', 'F1', 'faithfulness (gpt-3.5-turbo-instruct)', 'answer_relevancy (gpt-3.5-turbo-instruct)', 'ragas_score (gpt-3.5-turbo-instruct)'],
    num_rows: 20
})

In [20]:
result_all.to_pandas()

,EM,F1,faithfulness (gpt-3.5-turbo-instruct),answer_relevancy (gpt-3.5-turbo-instruct),ragas_score (gpt-3.5-turbo-instruct)
0,1,1.000000,1.000000,0.839067,0.912492
1,1,1.000000,1.000000,0.787693,0.881240
2,1,1.000000,1.000000,0.827112,0.905377
3,1,1.000000,1.000000,0.769419,0.869686
4,1,1.000000,1.000000,0.848734,0.918178
5,0,0.199367,1.000000,0.934078,0.965915
6,0,0.356436,1.000000,0.938330,0.968184
7,0,0.383495,1.000000,0.960979,0.980101
8,0,0.671642,1.000000,0.932541,0.965093
9,0,0.331288,1.000000,0.915574,0.955927


In [21]:
from langchain.chat_models import ChatOpenAI
from ragas.metrics import Faithfulness, AnswerRelevancy

gpt3 = ChatOpenAI(model_name="gpt-3.5-turbo")
faithfulness_gpt3 = Faithfulness(name="faithfulness", llm=gpt3)
answer_relevancy_gpt3 = AnswerRelevancy(name="answer_relevancy", llm=gpt3)

gpt4 = ChatOpenAI(model_name="gpt-4")
faithfulness_gpt4 = Faithfulness(name="faithfulness", llm=gpt4)
answer_relevancy_gpt4 = AnswerRelevancy(name="answer_relevancy", llm=gpt4)

In [22]:
%%time
# evaluate

result_gpt3 = evaluate(
    pruned_ds,
    metrics=[
        faithfulness_gpt3,
        answer_relevancy_gpt3,
    ],
)

result_gpt3

evaluating with [faithfulness]


100%|██████████| 2/2 [04:09<00:00, 124.63s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [00:51<00:00, 25.66s/it]


CPU times: user 243 ms, sys: 52 ms, total: 295 ms
Wall time: 5min 1s


{'ragas_score': 0.8477, 'faithfulness': 0.7786, 'answer_relevancy': 0.9302}

In [23]:
df = result_gpt3.to_pandas()
df

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,1.000000,0.883615
1,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.800000,0.928607
2,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,\nApplying for and receiving business credit c...,"[""I'm afraid the great myth of limited liabili...",0.636364,0.919426
3,Intentions of Deductible Amount for Small Busi...,"[""Short answer, yes. But this is not done thro...",\nThe intention of deductible amounts for smal...,"[""If your sole proprietorship losses exceed al...",0.800000,0.904498
4,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...,0.666667,0.962661
5,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...",\nUsing credit card points to pay for tax dedu...,"[""For simplicity, let's start by just consider...",0.750000,0.912459
6,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...",\nInvesting/business with other people's money...,"[""Basically, you either borrow money, or get o...",1.000000,0.947913
7,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...",\nThere are several approaches for pricing a s...,"[I don't have any experience in this, but this...",1.000000,0.977382
8,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,\nMoney earned and spent prior to establishing...,[Funds earned and spent before opening a dedic...,1.000000,0.919864
9,Do I need a new EIN since I am hiring employee...,[You don't need to notify the IRS of new membe...,"\nNo, you do not need a new EIN since you are ...",[I called the IRS (click here for IRS contact ...,0.500000,0.994253


In [24]:
result_all = result_all.map(
    lambda record, idx: {
        "faithfulness (gpt-3.5-turbo)": df["faithfulness"][idx], 
        "answer_relevancy (gpt-3.5-turbo)": df["answer_relevancy"][idx], 
        "ragas_score (gpt-3.5-turbo)": 2 * df["faithfulness"][idx] * df["answer_relevancy"][idx] / (df["faithfulness"][idx] + df["answer_relevancy"][idx])
    },
    batched=False,
    with_indices=True
)
result_all

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['EM', 'F1', 'faithfulness (gpt-3.5-turbo-instruct)', 'answer_relevancy (gpt-3.5-turbo-instruct)', 'ragas_score (gpt-3.5-turbo-instruct)', 'faithfulness (gpt-3.5-turbo)', 'answer_relevancy (gpt-3.5-turbo)', 'ragas_score (gpt-3.5-turbo)'],
    num_rows: 20
})

In [25]:
result_all.to_pandas()

,EM,F1,faithfulness (gpt-3.5-turbo-instruct),answer_relevancy (gpt-3.5-turbo-instruct),ragas_score (gpt-3.5-turbo-instruct),faithfulness (gpt-3.5-turbo),answer_relevancy (gpt-3.5-turbo),ragas_score (gpt-3.5-turbo)
0,1,1.000000,1.000000,0.839067,0.912492,1.000000,0.883615,0.938212
1,1,1.000000,1.000000,0.787693,0.881240,0.800000,0.928607,0.859519
2,1,1.000000,1.000000,0.827112,0.905377,0.636364,0.919426,0.752144
3,1,1.000000,1.000000,0.769419,0.869686,0.800000,0.904498,0.849046
4,1,1.000000,1.000000,0.848734,0.918178,0.666667,0.962661,0.787778
5,0,0.199367,1.000000,0.934078,0.965915,0.750000,0.912459,0.823292
6,0,0.356436,1.000000,0.938330,0.968184,1.000000,0.947913,0.973260
7,0,0.383495,1.000000,0.960979,0.980101,1.000000,0.977382,0.988562
8,0,0.671642,1.000000,0.932541,0.965093,1.000000,0.919864,0.958260
9,0,0.331288,1.000000,0.915574,0.955927,0.500000,0.994253,0.665385


In [26]:
%%time
# evaluate

result_gpt4 = evaluate(
    pruned_ds,
    metrics=[
        faithfulness_gpt4,
        answer_relevancy_gpt4,
    ],
)

result_gpt4

evaluating with [faithfulness]


100%|██████████| 2/2 [13:25<00:00, 402.69s/it]


evaluating with [answer_relevancy]


100%|██████████| 2/2 [01:13<00:00, 36.56s/it]


CPU times: user 338 ms, sys: 0 ns, total: 338 ms
Wall time: 14min 39s


{'ragas_score': 0.8110, 'faithfulness': 0.7135, 'answer_relevancy': 0.9394}

In [27]:
df = result_gpt4.to_pandas()
df

,question,contexts,answer,ground_truths,faithfulness,answer_relevancy
0,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,0.833333,0.928281
1,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.400000,0.941752
2,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,\nApplying for and receiving business credit c...,"[""I'm afraid the great myth of limited liabili...",1.000000,0.920309
3,Intentions of Deductible Amount for Small Busi...,"[""Short answer, yes. But this is not done thro...",\nThe intention of deductible amounts for smal...,"[""If your sole proprietorship losses exceed al...",0.666667,0.907101
4,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...,0.666667,0.928894
5,Using credit card points to pay for tax deduct...,"[""For simplicity, let's start by just consider...",\nUsing credit card points to pay for tax dedu...,"[""For simplicity, let's start by just consider...",0.833333,0.919837
6,Investing/business with other people's money: ...,"[""Basically, you either borrow money, or get o...",\nInvesting/business with other people's money...,"[""Basically, you either borrow money, or get o...",1.000000,0.941194
7,What approaches are there for pricing a small ...,"[I don't have any experience in this, but this...",\nThere are several approaches for pricing a s...,"[I don't have any experience in this, but this...",0.750000,0.977345
8,How to account for money earned and spent prio...,[Funds earned and spent before opening a dedic...,\nMoney earned and spent prior to establishing...,[Funds earned and spent before opening a dedic...,1.000000,0.945639
9,Do I need a new EIN since I am hiring employee...,[You don't need to notify the IRS of new membe...,"\nNo, you do not need a new EIN since you are ...",[I called the IRS (click here for IRS contact ...,1.000000,0.988146


In [28]:
result_all = result_all.map(
    lambda record, idx: {
        "faithfulness (gpt-4)": df["faithfulness"][idx], 
        "answer_relevancy (gpt-4)": df["answer_relevancy"][idx], 
        "ragas_score (gpt-4)": 2 * df["faithfulness"][idx] * df["answer_relevancy"][idx] / (df["faithfulness"][idx] + df["answer_relevancy"][idx])
    },
    batched=False,
    with_indices=True
)
result_all

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['EM', 'F1', 'faithfulness (gpt-3.5-turbo-instruct)', 'answer_relevancy (gpt-3.5-turbo-instruct)', 'ragas_score (gpt-3.5-turbo-instruct)', 'faithfulness (gpt-3.5-turbo)', 'answer_relevancy (gpt-3.5-turbo)', 'ragas_score (gpt-3.5-turbo)', 'faithfulness (gpt-4)', 'answer_relevancy (gpt-4)', 'ragas_score (gpt-4)'],
    num_rows: 20
})

In [29]:
result_all.to_pandas()

,EM,F1,faithfulness (gpt-3.5-turbo-instruct),answer_relevancy (gpt-3.5-turbo-instruct),ragas_score (gpt-3.5-turbo-instruct),faithfulness (gpt-3.5-turbo),answer_relevancy (gpt-3.5-turbo),ragas_score (gpt-3.5-turbo),faithfulness (gpt-4),answer_relevancy (gpt-4),ragas_score (gpt-4)
0,1,1.000000,1.000000,0.839067,0.912492,1.000000,0.883615,0.938212,0.833333,0.928281,0.878248
1,1,1.000000,1.000000,0.787693,0.881240,0.800000,0.928607,0.859519,0.400000,0.941752,0.561506
2,1,1.000000,1.000000,0.827112,0.905377,0.636364,0.919426,0.752144,1.000000,0.920309,0.958501
3,1,1.000000,1.000000,0.769419,0.869686,0.800000,0.904498,0.849046,0.666667,0.907101,0.768517
4,1,1.000000,1.000000,0.848734,0.918178,0.666667,0.962661,0.787778,0.666667,0.928894,0.776232
5,0,0.199367,1.000000,0.934078,0.965915,0.750000,0.912459,0.823292,0.833333,0.919837,0.874451
6,0,0.356436,1.000000,0.938330,0.968184,1.000000,0.947913,0.973260,1.000000,0.941194,0.969706
7,0,0.383495,1.000000,0.960979,0.980101,1.000000,0.977382,0.988562,0.750000,0.977345,0.848712
8,0,0.671642,1.000000,0.932541,0.965093,1.000000,0.919864,0.958260,1.000000,0.945639,0.972060
9,0,0.331288,1.000000,0.915574,0.955927,0.500000,0.994253,0.665385,1.000000,0.988146,0.994038


In [30]:
pruned_ds.to_csv("pruned-ds.csv", sep='\t')
result_all.to_csv("pruned-result.csv", sep='\t')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3324